In [1]:
import scipy
import scipy.stats
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

In [3]:
df = pd.read_pickle('firehoseTAZ')

In [5]:
df['minute_of_day'] = df['time'].dt.hour*60+df['time'].dt.minute

In [6]:
df.head()

,time,cate,TAZ,minute_of_day
0,2017-01-01 00:02:56,Food,95,2
1,2017-01-01 00:07:38,Professional & Other Places,95,7
2,2017-01-01 00:19:20,Food,95,19
3,2017-01-01 00:19:25,Food,95,19
4,2017-01-01 00:21:18,Nightlife Spot,95,21


In [26]:
w=10 #sequenceLength
timeLimit=120 #the time window upbound for analyzing the farthest check-in sequence. here I set it as 2 hour for demo
n=318 #number of zones
T=1440 #i use one minute as temporal resolution
r_threshold=0.39

In [14]:
arrival_t=np.zeros((n,T))

In [15]:
for i in range(len(df)):
    arrival_t[df.loc[i,'TAZ']-1][df.loc[i,'minute_of_day']]+=1

In [16]:
arrival_t_pad=np.asarray([np.append(x,x[:w+timeLimit]) for x in arrival_t])

In [17]:
arrival_t_pad.shape

(318, 1570)

In [19]:
sequence_t=[]
for i in range(n):
    for t in range(T+timeLimit):
        sequence_t.append(arrival_t_pad[i][t:t+w])    

In [27]:
def frictionGene(w,timelimit,n,T,sequence_t,r_threshold):
    friction=np.zeros([n,n,T,T]) #we need to have both departure time and arrival time for time foot of dynamic OD estimation
    for i in range(n):
        for t in range(T):
            if np.var(sequence_t[i*T+t])!=0:
                x=sequence_t[i*timeCount+t]
                for j in range(n):
                    for d in range(t,t+timelimit):
                        if np.var(sequence_t[j*T+d])!=0:
                            y=sequence_t[j*T+d]
                            r=np.nan_to_num(scipy.stats.pearsonr(x,y))
                            friction[i][t][j][d]=np.where(abs(r)>=r_threshold,abs(r),0)
    return friction

In [30]:
R=frictionGene(w,timeLimit,n,T,sequence_t,r_threshold)

### now we have the zone correlation matrix